# <b>背景</b>

<p>ET（essential tremor）患者：原发性震颤(Essential tremor)患者，特发性震颤（essential tremor，ＥＴ）最常见的运动障碍性疾病，主要为手、头部及身体其他部位的姿位性和运动性震颤。发病部位：上肢、头、面部、下颚。</p>
<p>参见文献： louis2003 Factors associated with increased risk of head tremor in essential tremor_ a community-based study in northern Manhattan）。</p>
<p>中线震颤（midline tremor）：   包含：面部（下颌部+唇部）、舌头、声音、头部（又称颈部）和躯干。<p>

    
# <b>目的</b>
1.1.	探索ET患者伴中线震颤的危险因素。
    
1.2.	ET患者伴焦虑和抑郁的危险因素    
    
# <b>任务</b>

这份数据来自内科，手工输入导致数据缺失严重或中英文混乱等问题，数据清理工作较为繁琐。

# <b>第一步 引包</b>

In [40]:
#引包：引入所需python包
import xlrd
import os
import re
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve,auc
import matplotlib as mpl
from matplotlib import pyplot as plt
from numpy import nan
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

# <b>第二步 读取excel数据</b>

1）设置默认目录

2）通过表单名查找数据

3）读取该表单中所有数据

In [41]:
dir = "./"
print(os.listdir(dir))

['.ipynb_checkpoints', '1.数据读取和清洗.ipynb', 'data', 'output', '2.中线震颤并发情况.ipynb', '3.人口特征统计.ipynb', '4.疾病特征统计.ipynb', '5.变量相关性和重要性.ipynb', '6.危险因子（逻辑回归）.ipynb', 'requirements.txt', 'pip_search_result.txt']


In [42]:
# 读取excel数据
df = pd.DataFrame()
file = "data/ET虚构数据.xlsx"
data = xlrd.open_workbook(dir+file).sheet_by_name("总表")
df = pd.read_excel(dir+file,data.name,index_col=None,na_values=['9999'])
df.shape

(200, 57)

# <b>第三步 探索数据 </b>
<p>了解数据分布和数据质量</p>
<b>1) 查看字段名</b>

In [43]:
df.columns   #查看字段名

Index(['编号', '性别', '性别(0=女，1=男)', '工作状态合并栏（1=在职，2=无业）', '婚姻状况合并栏（1=已婚 2=单身）',
       '年龄', '发病年龄',
       '主要受累部位（无中线仅四肢=0，中线为主或先中线后四肢=1，四肢为主（先四肢后中线或同时出现但以四肢为主）=2）', '总病程',
       '上肢病程', '下肢病程', '颈部病程', '声音病程', '面部病程', '舌病程', '躯干病程', '主观认知功能下降',
       '受教育时间', 'MMSE（文盲>17分，小学>20分，初中及以上>24分）', '家族史', '高血压', '糖尿病', '其他',
       '抗ET药物使用', '抗焦虑抑郁药物使用', '吸烟', '饮酒', '静止性上肢震颤分数（TRS A 静止性）',
       '静止性下肢震颤分数（TRS A 下肢静止性震颤）', '静止性四肢震颤分数',
       '运动性上肢震颤总分（（TRS A运动性上肢+TRS B））', '运动性下肢震颤分数（TRS A 下肢运动性震颤）',
       '运动性四肢震颤总分', 'TRS C:15-22', '自述焦虑时长', 'HAMA总', '焦虑分类(0≤14，1≥15)',
       'HAMA3级（＜7=0级,7-14=1级，≥15=2级）', '自述抑郁时长', 'HAMD总分', '抑郁分类（0级＜21，1级≥21）',
       '3级（＜8=0,8-20=1，≥21=2）', '匹兹堡总分', '上肢静止性震颤（0=无，1=有）',
       '下肢静止性震颤（0=无，1=有）', '四肢静止性震颤', '面部（是=1，无=1）', '声音（是=1，无=0）',
       '颈部（是=1，无=0%）', '面声颈部位分级分数（0级=无；1级=1部位；2级=≥2部位）', '面声颈量表分数（单分）',
       '面声颈有无（0=无，1=有）', '声颈量表部位分级（0级=无；1级=1部位；2级=2部位）', '声颈量表分数（单分）', '声颈有无',
       '下肢震颤(0=无，1=有)', '意向性震颤'],
      dtype='object')

<p><b>发现：</b></p>
<p>---字段名有重复; </p>
<p>---字段名中带了中英文符号和注解，不方便后面使用列名。 </p>
<p><b>2) 查看数据类型 </b></p>

In [44]:
#-------DataType of columns
#print(df.dtypes)  #  [25:55]       #view  the columns types           
dtype_df = df.dtypes.reset_index()    # calculate the types stat 
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index() 

,Column Type,Count
0,int64,35
1,float64,3
2,object,19


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 57 columns):
编号                                                          200 non-null object
性别                                                          200 non-null object
性别(0=女，1=男)                                                 200 non-null int64
工作状态合并栏（1=在职，2=无业）                                          200 non-null int64
婚姻状况合并栏（1=已婚 2=单身）                                          200 non-null int64
年龄                                                          200 non-null int64
发病年龄                                                        200 non-null int64
主要受累部位（无中线仅四肢=0，中线为主或先中线后四肢=1，四肢为主（先四肢后中线或同时出现但以四肢为主）=2）    200 non-null int64
总病程                                                         200 non-null float64
上肢病程                                                        199 non-null object
下肢病程                                                        88 non-null object
颈部病程                 

<b>3) 查看值域</b>

In [46]:
df1 = df.iloc[:,2:]
for i in df1.columns:
    uniquevalue = str(df1[i].unique())
    print (i,":  ",uniquevalue)

性别(0=女，1=男) :   [0 1]
工作状态合并栏（1=在职，2=无业） :   [1 2]
婚姻状况合并栏（1=已婚 2=单身） :   [1 2]
年龄 :   [72 45 71 57 66 76 43 37 75 86 60 73 65 50 77 33 84 46 68 79 81 78 44 42
 80 74 62 55 69 61 32 34 52 31 70 41 59 88 54 85 35 56 40 82 83 53 30 48
 90 47 25 39 51 27 64 49 24 23 58 28 36 20 63 26]
发病年龄 :   [68 40 50 51 48 56 30 27 59 71 55 69 72 17 67 43 63 76 62 66 73 41 75 38
 34 23 42 60 45 83 65 29 20 49 25 14 74 11 31 85 24 64 61 58 84 53 36 32
 78 57 18 87 39 47 21 22 15 54 37 52 46 44 12 33 80 16  6]
主要受累部位（无中线仅四肢=0，中线为主或先中线后四肢=1，四肢为主（先四肢后中线或同时出现但以四肢为主）=2） :   [0 1 2 3]
总病程 :   [ 4.   5.  21.   6.  18.  20.  13.  10.  16.  15.   8.   3.  17.  40.
 25.  11.  51.   7.   2.5 14.  36.  30.  39.   9.  34.  28.  50.  33.
 12.  23. ]
上肢病程 :   [4 '查体见' 21 6 18 '2月' 10 16 15 5 '8+月' 20 8 3 '6月' 40 25 11 51 7 '3' '2.5'
 14 36 30 1 13 39 24 34 28 50 2 '12' 9 '5' 12 17 23 nan '3+月']
下肢病程 :   ['查体见' nan 10 7 4 '自述，但起病时间不详' '6+月' 1 2 13 20 '1' 3 5 0]
颈部病程 :   [nan 3 20 13 1 5 2 17 15 6 '查体见' '1+月' 10 '6+月' 7

<p><b> 发现 </b></p>
--病程字段：有“xx月”，空值，“查体见”，'自述，但起病时间不详'等取值，影响字段转成数值型继续进行分析；
--

<b>4）数据缺失情况 </b>

In [47]:
#missing data
print ("-----------------------------总记录数： " ,df.index.size,"条    --------------------------")
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['missing_count', 'missing_Percent'])
missing_data.head(20)

-----------------------------总记录数：  200 条    --------------------------


,missing_count,missing_Percent
躯干病程,197,0.985
下肢静止性震颤（0=无，1=有）,190,0.950
面部病程,182,0.910
舌病程,181,0.905
上肢静止性震颤（0=无，1=有）,168,0.840
声音病程,131,0.655
颈部病程,124,0.620
下肢病程,112,0.560
自述抑郁时长,2,0.010
自述焦虑时长,2,0.010


<p><b>发现：</b></p>
<p>---xx病程缺失较多：缺失值为未知病程或未出现该震颤; </p>
<p>---xx震颤：缺失值为未出现该震颤。 </p>

# <b>第四步 预处理 </b>
数据清理/预处理是一个非常繁琐的工作，根据业务需求和数据质量而不同，没有标准的流程。下面根据实列来了解常用的一些操作：

<b>1）删除重复字段</b>

从“检查日期”中提取年份信息

In [48]:
x = [1]  # 两个 性别 字段 重复了
cols = df.columns[x]
print (" 重复列,共计",len(cols),"个字段:    " , cols)
df.drop(cols,axis=1,inplace=True)

 重复列,共计 1 个字段:     Index(['性别'], dtype='object')


<b> 2）删除字段名称中无用字符 </b>

In [49]:
df.columns = [re.sub('（.*?）', '', hdr)  for hdr in df.columns]  #删除 中文括号及括号内字符
df.columns = [re.sub(u"\\(.*?\\)|\\{.*?}|\\[.*?]", '', hdr)  for hdr in df.columns]  #删除 英文(半角)符号及符号内字符
df.columns = [re.sub("）", '', hdr)  for hdr in df.columns]   #删除 未匹配的中文括号
print("data rows and columns : ", df.shape )    #209条数据，57个字段

data rows and columns :  (200, 56)


In [50]:
df.columns

Index(['编号', '性别', '工作状态合并栏', '婚姻状况合并栏', '年龄', '发病年龄', '主要受累部位=2', '总病程',
       '上肢病程', '下肢病程', '颈部病程', '声音病程', '面部病程', '舌病程', '躯干病程', '主观认知功能下降',
       '受教育时间', 'MMSE', '家族史', '高血压', '糖尿病', '其他', '抗ET药物使用', '抗焦虑抑郁药物使用',
       '吸烟', '饮酒', '静止性上肢震颤分数', '静止性下肢震颤分数', '静止性四肢震颤分数', '运动性上肢震颤总分',
       '运动性下肢震颤分数', '运动性四肢震颤总分', 'TRS C:15-22', '自述焦虑时长', 'HAMA总', '焦虑分类',
       'HAMA3级', '自述抑郁时长', 'HAMD总分', '抑郁分类', '3级', '匹兹堡总分', '上肢静止性震颤',
       '下肢静止性震颤', '四肢静止性震颤', '面部', '声音', '颈部', '面声颈部位分级分数', '面声颈量表分数', '面声颈有无',
       '声颈量表部位分级', '声颈量表分数', '声颈有无', '下肢震颤', '意向性震颤'],
      dtype='object')

<b> 3) 病程字段清洗 </b>


In [51]:
# 替换数值型缺失值为零
df.loc[:,["总病程", "上肢病程", "下肢病程","颈部病程", "声音病程", "面部病程", "舌病程", "躯干病程"]].fillna(0)
df.head(2)

,编号,性别,工作状态合并栏,婚姻状况合并栏,年龄,发病年龄,主要受累部位=2,总病程,上肢病程,下肢病程,...,声音,颈部,面声颈部位分级分数,面声颈量表分数,面声颈有无,声颈量表部位分级,声颈量表分数,声颈有无,下肢震颤,意向性震颤
0,G001,0,1,1,72,68,0,4.0,4,查体见,...,0,0,0,0,0,0,0,0,1,0
1,G002,0,1,1,45,40,1,5.0,查体见,查体见,...,1,1,2,3,1,2,3,1,1,0


In [52]:
#替换字符串缺失值为零
for i in ["总病程", "上肢病程", "下肢病程","颈部病程", "声音病程", "面部病程", "舌病程", "躯干病程"]:
    df.loc[df[i].astype(str)=="",i] =0
    df.loc[df[i].astype(str) =="nan",i] = 0
    df.loc[df[i].astype(str) == "查体见",i] = ""   #临时补充的，后期可能有其他方式处理
df.head(2)

,编号,性别,工作状态合并栏,婚姻状况合并栏,年龄,发病年龄,主要受累部位=2,总病程,上肢病程,下肢病程,...,声音,颈部,面声颈部位分级分数,面声颈量表分数,面声颈有无,声颈量表部位分级,声颈量表分数,声颈有无,下肢震颤,意向性震颤
0,G001,0,1,1,72,68,0,4,4,,...,0,0,0,0,0,0,0,0,1,0
1,G002,0,1,1,45,40,1,5,,,...,1,1,2,3,1,2,3,1,1,0


In [53]:
for i in ["总病程", "上肢病程", "下肢病程","颈部病程", "声音病程", "面部病程", "舌病程", "躯干病程"]:
    
    df[i] = np.where(df[i].astype(str).str.contains(r"\d"),df[i],"")   #不含数值的字符串替换为空值
    df.loc[df[i].astype(str)=="",i] =0.001                             #将空值设置为任意值，例如0.001，避免字段转成数值型时，缺失值导致报错
    df[i] = np.where(df[i].astype(str).str.contains("月"),df[i].astype(str).apply(lambda x:str(float(re.sub("\D","",x))/12)),df[i])  #含“月”的取值，只取数值
    df[i] = df[i].astype(float)                                      # 将只含数值的列转成浮点数值型
df.head(2)

,编号,性别,工作状态合并栏,婚姻状况合并栏,年龄,发病年龄,主要受累部位=2,总病程,上肢病程,下肢病程,...,声音,颈部,面声颈部位分级分数,面声颈量表分数,面声颈有无,声颈量表部位分级,声颈量表分数,声颈有无,下肢震颤,意向性震颤
0,G001,0,1,1,72,68,0,4.0,4.000,0.001,...,0,0,0,0,0,0,0,0,1,0
1,G002,0,1,1,45,40,1,5.0,0.001,0.001,...,1,1,2,3,1,2,3,1,1,0


# <b>第五步 导出结果 </b>


In [54]:
df.to_csv(dir+"output/df_clean.csv")